In [1]:
# Unistall All

!pip uninstall -y transformers tokenizers trl
!rm -rf /usr/local/lib/python3.*/dist-packages/transformers*
!rm -rf /usr/local/lib/python3.*/dist-packages/tokenizers*
!rm -rf /usr/local/lib/python3.*/dist-packages/trl*
!pip cache purge
!pip uninstall -y transformers
!pip uninstall -y tokenizers
!pip uninstall -y unsloth unsloth_zoo huggingface_hub

Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: tokenizers 0.22.1
Uninstalling tokenizers-0.22.1:
  Successfully uninstalled tokenizers-0.22.1
Files removed: 0
Found existing installation: huggingface-hub 0.35.3
Uninstalling huggingface-hub-0.35.3:
  Successfully uninstalled huggingface-hub-0.35.3


In [2]:
# Install All

!pip install --upgrade huggingface_hub
from huggingface_hub import login
login()

!pip install --upgrade transformers datasets peft

!pip install datasets --upgrade

!pip install matplotlib-venn

!apt-get -qq install -y libfluidsynth1

!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

!apt-get -qq install -y graphviz && pip install pydot
import pydot

!pip install cartopy
import cartopy

!pip install transformers -U


!pip install transformers trl

!pip install unsloth


!python -m venv transformers_v5

!source transformers_v5/bin/activate


!pip install --upgrade pip

!pip install torch transformers datasets accelerate peft



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, which is not installed.
peft 0.17.1 requires transformers, which is not installed.
torchtune 0.6.1 requires tokenizers, which is not installed.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 140.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 119.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0

In [3]:

# ============================================================
# 🔹 #Import necessary Python libraries
# ============================================================

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

from trl import SFTTrainer
import re
import numpy as np
import urllib
from packaging import version
import tarfile
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import transformers, tokenizers, trl
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, classification_report
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, BitsAndBytesConfig, EarlyStoppingCallback
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import accuracy_score, f1_score
import itertools
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, classification_report
print("Transformers:", transformers.__version__)

Transformers: 4.56.2


In [4]:
# ============================================================
# 🔹 Load dataset directly from GitHub raw link
# ============================================================
csv_path = "https://raw.githubusercontent.com/suralk/travel_domain_data/master/5000TravelQuestionsDataset.csv"
df = pd.read_csv(csv_path, header=None, encoding="latin1")

# drop by column index (axis=1 means column)
df = df.drop(df.columns[2], axis=1)

print(df.shape)
print(df.head())


(5000, 2)
                                                   0    1
0  What are the special things we (husband and me...  TTD
1  What are the companies which organize shark fe...  TTD
2  Is it safe for female traveller to go alone to...  TGU
3  What are the best places around Cape Town for ...  TTD
4  What are the best places to stay for a family ...  ACM


In [5]:
# ============================================================
# 🔹 Shuffle, Split Travel Question Dataset
# ============================================================

# -----------------------------
#  Load Dataset from URL
# -----------------------------
csv_url = "https://raw.githubusercontent.com/suralk/travel_domain_data/master/5000TravelQuestionsDataset.csv"

# Use latin1 encoding to handle special characters
df = pd.read_csv(csv_url, header=None, encoding="latin1")

# Drop unnecessary 3rd column if it exists
if df.shape[1] > 2:
    df = df.drop(df.columns[2], axis=1)

df.columns = ["question", "label"]

print(f"Total dataset size: {len(df)} rows")

# -----------------------------
#  Shuffle Dataset
# -----------------------------
seed = 42
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)

# -----------------------------
#  Split Dataset (4000 / 700 / 300)
# -----------------------------
N_required = 4000 + 700 + 300
if len(df_shuffled) < N_required:
    raise ValueError(f"Dataset too small: need at least {N_required} rows, found {len(df_shuffled)}")

df_selected = df_shuffled.iloc[:N_required].reset_index(drop=True)

train_df = df_selected.iloc[:4000].reset_index(drop=True)
test_df  = df_selected.iloc[4000:4000+700].reset_index(drop=True)
val_df   = df_selected.iloc[4000+700:4000+700+300].reset_index(drop=True)

print("Split sizes -> Train:", len(train_df), "| Test:", len(test_df), "| Val:", len(val_df))





Total dataset size: 5000 rows
Split sizes -> Train: 4000 | Test: 700 | Val: 300


In [6]:
# ============================================================
#  Save CSVs locally
# ============================================================
train_path = "train_dataset.csv"
test_path  = "test_dataset.csv"
val_path   = "val_dataset.csv"

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)
val_df.to_csv(val_path, index=False)

print(f"CSV files saved locally: {train_path}, {test_path}, {val_path}")

# -----------------------------
#  Download
# -----------------------------
try:
    # Check if running in Colab
    from google.colab import files
    files.download(train_path)
    files.download(test_path)
    files.download(val_path)
except ImportError:
    # Not Colab → provide clickable download links
    try:
        from IPython.display import FileLink, display
        print("Not running in Colab. Use the links below to download the CSVs:")
        display(FileLink(train_path))
        display(FileLink(test_path))
        display(FileLink(val_path))
    except:
        print("Download links not available. CSVs are saved in the current directory:", os.getcwd())

CSV files saved locally: train_dataset.csv, test_dataset.csv, val_dataset.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# ============================================================
# Define the Model name
# ============================================================

model_name = "unsloth/zephyr-sft"

from transformers import AutoConfig, AutoTokenizer
from huggingface_hub import HfApi

config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
api = HfApi()
card = api.model_info(model_name)
metadata = card.cardData


# Basic info
print("Model type:", config.model_type)
print("Hidden size:", getattr(config, "hidden_size", "N/A"))
print("Number of layers:", getattr(config, "num_hidden_layers", "N/A"))
print("Number of attention heads:", getattr(config, "num_attention_heads", "N/A"))
print("Activation function:", getattr(config, "hidden_act", "N/A"))
print("Positional embedding type:", getattr(config, "rope_scaling", "RoPE / sinusoidal"))
print("Context length (max tokens):", getattr(config, "max_position_embeddings", "N/A"))
print("Tokenizer vocab size:", tokenizer.vocab_size)
print("Transformers:", transformers.__version__)
print("Tokenizers:", tokenizers.__version__)
print("TRL:", trl.__version__)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

Model type: mistral
Hidden size: 4096
Number of layers: 32
Number of attention heads: 32
Activation function: silu
Positional embedding type: RoPE / sinusoidal
Context length (max tokens): 32768
Tokenizer vocab size: 32000
Transformers: 4.56.2
Tokenizers: 0.22.1
TRL: 0.23.0


In [9]:
# ============================================================
# 🔹 Zero-Shot Prompting on Train Dataset
# ============================================================


import pandas as pd
import torch, re, gc
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, classification_report

torch.backends.cudnn.benchmark = True

# -----------------------------
# 1. Load Model & Tokenizer
# -----------------------------
model_name = "unsloth/zephyr-sft"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

# -----------------------------
# 2. Load Train Dataset
# -----------------------------
train_df = pd.read_csv("train_dataset.csv").reset_index(drop=True)

fine_to_coarse = {
    "TTD": "TRAVEL", "TGU": "TRAVEL", "TRS": "TRAVEL",
    "ACM": "STAY",
    "FOD": "EXPERIENCE", "ENT": "EXPERIENCE",
    "WTH": "INFORMATION"
}

def map_to_coarse(label):
    code = ''.join([c for c in str(label) if c.isalpha()])[:3].upper()
    return fine_to_coarse.get(code, "INFORMATION")

train_df["coarse_label"] = train_df["label"].apply(map_to_coarse)

# -----------------------------
# 3. Build Definition-Enhanced Zero-Shot Prompt
# -----------------------------
def build_zero_shot_prompt(question):
    return (
        "You are an expert travel question classifier.\n"
        "Classify the following travel-related question into one of these categories:\n\n"
        "TRAVEL: transportation, sightseeing, routes, trip planning, tours.\n"
        "STAY: hotels, accommodation, lodging.\n"
        "EXPERIENCE: food, entertainment, local culture, activities, festivals.\n"
        "INFORMATION: weather, safety, general guidance, timing, or visa info.\n\n"
        f"Question: {question}\n"
        "Answer with only one category name: TRAVEL, STAY, EXPERIENCE, or INFORMATION.\n"
        "Category:"
    )

# -----------------------------
# 4. Batched Inference for Speed
# -----------------------------
def run_zero_shot_inference(df, batch_size=12):
    predictions = []
    for i in tqdm(range(0, len(df), batch_size), desc="Generating Zero-Shot Predictions"):
        batch_questions = df["question"].iloc[i:i+batch_size].tolist()
        prompts = [build_zero_shot_prompt(q) for q in batch_questions]

        # Tokenize in batch
        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=15,
                temperature=0.3,
                top_p=0.9,
                do_sample=True,   # slightly stochastic for zero-shot
                repetition_penalty=1.0
            )

        decoded = [tokenizer.decode(o, skip_special_tokens=True).upper() for o in outputs]

        for r in decoded:
            match = re.search(r"\b(TRAVEL|STAY|EXPERIENCE|INFORMATION)\b", r)
            label = match.group(1) if match else "INFORMATION"
            predictions.append(label)

        # Free memory per batch
        del inputs, outputs
        torch.cuda.empty_cache()
        gc.collect()

    return predictions

# -----------------------------
# 5. Run Zero-Shot Prompting
# -----------------------------
train_df["predicted_label_zeroshot"] = run_zero_shot_inference(train_df)

# -----------------------------
# 6. Evaluate
# -----------------------------
accuracy = accuracy_score(train_df["coarse_label"], train_df["predicted_label_zeroshot"])
print(f"\n✅ Zero-Shot Accuracy on Train Dataset: {round(accuracy, 2)}")

print("\nClassification Report:\n")
print(classification_report(
    train_df["coarse_label"], train_df["predicted_label_zeroshot"], zero_division=0
))

print("\nSample Predictions:\n")
print(train_df.sample(10)[["question", "coarse_label", "predicted_label_zeroshot"]])


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Zero-Shot Predictions:   6%|▋         | 21/334 [08:55<2:12:56, 25.48s/it]


KeyboardInterrupt: 

In [ ]:
# ============================================================
# 🔹 Zero-Shot Prompting on Test Dataset
# ============================================================

import pandas as pd
import torch, re, gc
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, classification_report

torch.backends.cudnn.benchmark = True

# -----------------------------
# 1. Load Model & Tokenizer
# -----------------------------
model_name = "unsloth/zephyr-sft"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

# -----------------------------
# 2. Load Test Dataset
# -----------------------------
test_df = pd.read_csv("test_dataset.csv").reset_index(drop=True)

fine_to_coarse = {
    "TTD": "TRAVEL", "TGU": "TRAVEL", "TRS": "TRAVEL",
    "ACM": "STAY",
    "FOD": "EXPERIENCE", "ENT": "EXPERIENCE",
    "WTH": "INFORMATION"
}

def map_to_coarse(label):
    code = ''.join([c for c in str(label) if c.isalpha()])[:3].upper()
    return fine_to_coarse.get(code, "INFORMATION")

test_df["coarse_label"] = test_df["label"].apply(map_to_coarse)

# -----------------------------
# 3. Build Definition-Enhanced Zero-Shot Prompt
# -----------------------------
def build_zero_shot_prompt(question):
    return (
        "You are an expert travel question classifier.\n"
        "Classify the following travel-related question into one of these categories:\n\n"
        "TRAVEL: transportation, sightseeing, routes, trip planning, tours.\n"
        "STAY: hotels, accommodation, lodging.\n"
        "EXPERIENCE: food, entertainment, local culture, activities, festivals.\n"
        "INFORMATION: weather, safety, general guidance, timing, or visa info.\n\n"
        f"Question: {question}\n"
        "Answer with only one category name: TRAVEL, STAY, EXPERIENCE, or INFORMATION.\n"
        "Category:"
    )

# -----------------------------
# 4. Batched Inference for Speed
# -----------------------------
def run_zero_shot_inference(df, batch_size=12):
    predictions = []
    for i in tqdm(range(0, len(df), batch_size), desc="Generating Zero-Shot Predictions"):
        batch_questions = df["question"].iloc[i:i+batch_size].tolist()
        prompts = [build_zero_shot_prompt(q) for q in batch_questions]

        # Tokenize in batch
        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=15,
                temperature=0.3,
                top_p=0.9,
                do_sample=True,   # deterministic for zero-shot
                repetition_penalty=1.0
            )

        decoded = [tokenizer.decode(o, skip_special_tokens=True).upper() for o in outputs]

        for r in decoded:
            match = re.search(r"\b(TRAVEL|STAY|EXPERIENCE|INFORMATION)\b", r)
            label = match.group(1) if match else "INFORMATION"
            predictions.append(label)

        # Free memory per batch
        del inputs, outputs
        torch.cuda.empty_cache()
        gc.collect()

    return predictions

# -----------------------------
# 5. Run Zero-Shot Prompting
# -----------------------------
test_df["predicted_label_zeroshot"] = run_zero_shot_inference(test_df)

# -----------------------------
# 6. Evaluate
# -----------------------------
accuracy = accuracy_score(test_df["coarse_label"], test_df["predicted_label_zeroshot"])
print(f"\n✅ Zero-Shot Accuracy on Test Dataset: {round(accuracy, 2)}")

print("\nClassification Report:\n")
print(classification_report(
    test_df["coarse_label"], test_df["predicted_label_zeroshot"], zero_division=0
))

print("\nSample Predictions:\n")
print(test_df.sample(10)[["question", "coarse_label", "predicted_label_zeroshot"]])


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Zero-Shot Predictions: 100%|██████████| 59/59 [01:14<00:00,  1.26s/it]


✅ Zero-Shot Accuracy on Test Dataset: 0.67

Classification Report:

              precision    recall  f1-score   support

  EXPERIENCE       0.00      0.00      0.00       103
 INFORMATION       0.00      0.00      0.00        13
        STAY       0.00      0.00      0.00       112
      TRAVEL       0.67      1.00      0.81       472

    accuracy                           0.67       700
   macro avg       0.17      0.25      0.20       700
weighted avg       0.45      0.67      0.54       700


Sample Predictions:

                                              question coarse_label  \
231  Where to stay in Upolu for a solo female trave...         STAY   
671  Where can i find a villa on Zakynthos that has...         STAY   
242  Does anyone know of any cool speakeasies in Sa...   EXPERIENCE   
561   How much would Everest Panorama Trek would cost?       TRAVEL   
604                   What to wear in December in CPH?       TRAVEL   
481  Is it safe from jelly fishes and other pois

In [ ]:
# ============================================================
# 🔹 1-Shot Prompt on Train Dataset
# ============================================================

import pandas as pd
import torch, re, gc
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModelForCausalLM

torch.backends.cudnn.benchmark = True

# -----------------------------
# 1. Load Model & Tokenizer
# -----------------------------
model_name = "unsloth/zephyr-sft"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

# -----------------------------
# 2. Load Train Dataset
# -----------------------------
train_df = pd.read_csv("train_dataset.csv").reset_index(drop=True)

# -----------------------------
# 3. Define Coarse Label Mapping
# -----------------------------
fine_to_coarse = {
    "TTD": "TRAVEL", "TGU": "TRAVEL", "TRS": "TRAVEL",
    "ACM": "STAY",
    "FOD": "EXPERIENCE", "ENT": "EXPERIENCE",
    "WTH": "INFORMATION"
}

def map_to_coarse(label):
    code = ''.join([c for c in str(label) if c.isalpha()])[:3].upper()
    return fine_to_coarse.get(code, "INFORMATION")

train_df["coarse_label"] = train_df["label"].apply(map_to_coarse)

# -----------------------------
# 4. Select 1–3 Sample Examples
# -----------------------------
# You can randomize or manually choose samples
examples = train_df.sample(3, random_state=42).to_dict(orient="records")

print("📌 Selected 1–3 Samples for 1-Shot Prompting:\n")
for i, ex in enumerate(examples, 1):
    print(f"{i}. Q: {ex['question']} → {ex['coarse_label']}")

# -----------------------------
# 5. Build Definition-Enhanced 1-Shot Prompt
# -----------------------------
def build_prompt(question, example):
    return (
        "You are an expert travel question classifier.\n"
        "Classify each question into one of these categories:\n"
        "TRAVEL: transportation, routes, travel planning, tours.\n"
        "STAY: hotels, accommodation, lodging.\n"
        "EXPERIENCE: activities, food, entertainment, local culture.\n"
        "INFORMATION: weather, safety, or general travel guidance.\n\n"
        f"Example:\nQ: {example['question']}\nA: {example['coarse_label']}\n\n"
        f"Now classify the following:\nQ: {question}\nA:"
    )

# -----------------------------
# 6. Function for 1-Shot Evaluation
# -----------------------------
def run_1shot(example, df, model, tokenizer, batch_size=12):
    predictions = []
    for i in tqdm(range(0, len(df), batch_size), desc="Generating"):
        batch_questions = df["question"].iloc[i:i+batch_size].tolist()
        prompts = [build_prompt(q, example) for q in batch_questions]
        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=10,
                temperature=0.2,
                top_p=0.9,
                do_sample=True
            )

        decoded = [tokenizer.decode(o, skip_special_tokens=True).upper() for o in outputs]
        for r in decoded:
            r = re.sub(r"[^A-Z\s]", "", r)
            match = re.search(r"\b(TRAVEL|STAY|EXPERIENCE|INFORMATION)\b", r)
            label = match.group(1) if match else "INFORMATION"
            predictions.append(label)

        del inputs, outputs
        torch.cuda.empty_cache()
        gc.collect()

    df["predicted_label"] = predictions
    acc = accuracy_score(df["coarse_label"], df["predicted_label"])
    return round(acc, 2), df

# -----------------------------
# 7. Run 1-Shot for Each Example
# -----------------------------
for i, example in enumerate(examples, 1):
    print(f"\n==============================")
    print(f"🧩 1-Shot Example {i}: {example['coarse_label']}")
    print(f"Q: {example['question']}\n")
    acc, df_pred = run_1shot(example, train_df.copy(), model, tokenizer)
    print(f"✅ Accuracy: {acc}")
    print(classification_report(df_pred["coarse_label"], df_pred["predicted_label"], zero_division=0))
    print(df_pred.sample(5)[["question", "coarse_label", "predicted_label"]])


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

📌 Selected 1–3 Samples for 1-Shot Prompting:

1. Q: I want to go shopping in Colombo, but do not want western sterile shops. Where should I go? → TRAVEL
2. Q: How to book Helicopter services for Kedarnath? → TRAVEL
3. Q: Where can I buy tickets for visit Kremlin? → TRAVEL

🧩 1-Shot Example 1: TRAVEL
Q: I want to go shopping in Colombo, but do not want western sterile shops. Where should I go?



Generating: 100%|██████████| 334/334 [05:39<00:00,  1.02s/it]


✅ Accuracy: 0.68
              precision    recall  f1-score   support

  EXPERIENCE       0.00      0.00      0.00       581
 INFORMATION       0.00      0.00      0.00       150
        STAY       0.00      0.00      0.00       567
      TRAVEL       0.68      1.00      0.81      2702

    accuracy                           0.68      4000
   macro avg       0.17      0.25      0.20      4000
weighted avg       0.46      0.68      0.54      4000

                                               question coarse_label  \
516    Can you suggest an itinerary for 5 days in Jeju?       TRAVEL   
522   Which transport method is the best to explore ...       TRAVEL   
2495     How disabled wheelchair friendly is Jerusalem?       TRAVEL   
495   which will be the best hotels to stay close to...         STAY   
1421  Who are the tour operators to organise a tour ...       TRAVEL   

     predicted_label  
516           TRAVEL  
522           TRAVEL  
2495          TRAVEL  
495           TRAVEL  


Generating: 100%|██████████| 334/334 [05:35<00:00,  1.00s/it]


✅ Accuracy: 0.68
              precision    recall  f1-score   support

  EXPERIENCE       0.00      0.00      0.00       581
 INFORMATION       0.00      0.00      0.00       150
        STAY       0.00      0.00      0.00       567
      TRAVEL       0.68      1.00      0.81      2702

    accuracy                           0.68      4000
   macro avg       0.17      0.25      0.20      4000
weighted avg       0.46      0.68      0.54      4000

                                               question coarse_label  \
2742           Advice needed about 2 night stay at GC ?         STAY   
1395       What is the temperature that time of month ?  INFORMATION   
3804  Am I correct in believing that I will need to ...       TRAVEL   
3233  What are some of the most romantic things to d...       TRAVEL   
3267         Has anyone travelled this route 25 by bus?       TRAVEL   

     predicted_label  
2742          TRAVEL  
1395          TRAVEL  
3804          TRAVEL  
3233          TRAVEL  


Generating: 100%|██████████| 334/334 [05:34<00:00,  1.00s/it]

✅ Accuracy: 0.68
              precision    recall  f1-score   support

  EXPERIENCE       0.00      0.00      0.00       581
 INFORMATION       0.00      0.00      0.00       150
        STAY       0.00      0.00      0.00       567
      TRAVEL       0.68      1.00      0.81      2702

    accuracy                           0.68      4000
   macro avg       0.17      0.25      0.20      4000
weighted avg       0.46      0.68      0.54      4000

                                               question coarse_label  \
197   What are the hotels with rooms that have a fir...         STAY   
436   Should I change currency in Eilat before cross...       TRAVEL   
1849  How much will it cost for a harbour bus trip a...       TRAVEL   
3452  Can we hire a private chef for reasonable pric...   EXPERIENCE   
2266    How many nights do you recommend to stay in FL?         STAY   

     predicted_label  
197           TRAVEL  
436           TRAVEL  
1849          TRAVEL  
3452          TRAVEL  


In [ ]:
# ============================================================
# 🔹 3-Shot Prompting on Train Dataset
# ============================================================

import pandas as pd
import torch, re, gc
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModelForCausalLM

torch.backends.cudnn.benchmark = True

# -----------------------------
# 1. Load Model & Tokenizer
# -----------------------------
model_name = "unsloth/zephyr-sft"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

# -----------------------------
# 2. Load and Prepare Train Dataset
# -----------------------------
train_df = pd.read_csv("train_dataset.csv").reset_index(drop=True)

fine_to_coarse = {
    "TTD": "TRAVEL", "TGU": "TRAVEL", "TRS": "TRAVEL",
    "ACM": "STAY",
    "FOD": "EXPERIENCE", "ENT": "EXPERIENCE",
    "WTH": "INFORMATION"
}

def map_to_coarse(label):
    code = ''.join([c for c in str(label) if c.isalpha()])[:3].upper()
    return fine_to_coarse.get(code, "INFORMATION")

train_df["coarse_label"] = train_df["label"].apply(map_to_coarse)

# -----------------------------
# 3. Select 1–3 Samples for 3-Shot Prompt
# -----------------------------
examples = train_df.sample(3, random_state=42).to_dict(orient="records")

print("📌 Selected 3 Examples for 3-Shot Prompting:\n")
for i, ex in enumerate(examples, 1):
    print(f"{i}. Q: {ex['question']} → {ex['coarse_label']}")
print("\n")

# -----------------------------
# 4. Build Definition-Enhanced 3-Shot Prompt
# -----------------------------
def build_3shot_prompt(question, examples):
    header = (
        "You are an expert travel question classifier.\n"
        "Classify each question into one of these categories:\n"
        "TRAVEL: transportation, routes, travel planning, tours.\n"
        "STAY: hotels, accommodation, lodging.\n"
        "EXPERIENCE: activities, food, entertainment, local culture.\n"
        "INFORMATION: weather, safety, or general travel guidance.\n\n"
        "Here are 3 examples:\n"
    )
    few_shot = ""
    for ex in examples:
        few_shot += f"Q: {ex['question']}\nA: {ex['coarse_label']}\n\n"
    task = f"Now classify the following question:\nQ: {question}\nA:"
    return header + few_shot + task

# -----------------------------
# 5. Batched Inference Function
# -----------------------------
def run_3shot_inference(df, examples, batch_size=12):
    predictions = []
    for i in tqdm(range(0, len(df), batch_size), desc="Generating 3-Shot Predictions"):
        batch_questions = df["question"].iloc[i:i+batch_size].tolist()
        prompts = [build_3shot_prompt(q, examples) for q in batch_questions]

        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=15,
                temperature=0.3,
                top_p=0.9,
                do_sample=True,
                repetition_penalty=1.0
            )

        decoded = [tokenizer.decode(o, skip_special_tokens=True).upper() for o in outputs]

        for r in decoded:
            match = re.search(r"\b(TRAVEL|STAY|EXPERIENCE|INFORMATION)\b", r)
            label = match.group(1) if match else "INFORMATION"
            predictions.append(label)

        del inputs, outputs
        torch.cuda.empty_cache()
        gc.collect()

    return predictions

# -----------------------------
# 6. Run 3-Shot Prompting
# -----------------------------
train_df["predicted_label_3shot"] = run_3shot_inference(train_df, examples)

# -----------------------------
# 7. Evaluate Performance
# -----------------------------
accuracy = accuracy_score(train_df["coarse_label"], train_df["predicted_label_3shot"])
print(f"\n✅ 3-Shot Accuracy on Train Dataset: {round(accuracy, 2)}")

print("\nClassification Report:\n")
print(classification_report(
    train_df["coarse_label"], train_df["predicted_label_3shot"], zero_division=0
))

print("\nSample Predictions:\n")
print(train_df.sample(10)[["question", "coarse_label", "predicted_label_3shot"]])


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

📌 Selected 3 Examples for 3-Shot Prompting:

1. Q: I want to go shopping in Colombo, but do not want western sterile shops. Where should I go? → TRAVEL
2. Q: How to book Helicopter services for Kedarnath? → TRAVEL
3. Q: Where can I buy tickets for visit Kremlin? → TRAVEL




Generating 3-Shot Predictions: 100%|██████████| 334/334 [07:14<00:00,  1.30s/it]


✅ 3-Shot Accuracy on Train Dataset: 0.68

Classification Report:

              precision    recall  f1-score   support

  EXPERIENCE       0.00      0.00      0.00       581
 INFORMATION       0.00      0.00      0.00       150
        STAY       0.00      0.00      0.00       567
      TRAVEL       0.68      1.00      0.81      2702

    accuracy                           0.68      4000
   macro avg       0.17      0.25      0.20      4000
weighted avg       0.46      0.68      0.54      4000


Sample Predictions:

                                               question coarse_label  \
2226  Is the entire month of January really busy or ...   EXPERIENCE   
1566  How much would be the cost for a sand art in D...       TRAVEL   
2279  What would be the average cost of a water bott...       TRAVEL   
3055  What is the best Italian restaurant in Playa B...   EXPERIENCE   
2659  Who is a good Florist for a wedding near lake ...       TRAVEL   
2083   Does Bora Bora have any regulations o

In [ ]:
# ============================================================
# 🔹 1-Shot Prompting on Test Dataset
# ============================================================

import pandas as pd
import torch, re, gc
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModelForCausalLM

torch.backends.cudnn.benchmark = True

# -----------------------------
# 1. Load Model & Tokenizer
# -----------------------------
model_name = "unsloth/zephyr-sft"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

# -----------------------------
# 2. Load and Prepare Test Dataset
# -----------------------------
test_df = pd.read_csv("test_dataset.csv").reset_index(drop=True)

fine_to_coarse = {
    "TTD": "TRAVEL", "TGU": "TRAVEL", "TRS": "TRAVEL",
    "ACM": "STAY",
    "FOD": "EXPERIENCE", "ENT": "EXPERIENCE",
    "WTH": "INFORMATION"
}

def map_to_coarse(label):
    code = ''.join([c for c in str(label) if c.isalpha()])[:3].upper()
    return fine_to_coarse.get(code, "INFORMATION")

test_df["coarse_label"] = test_df["label"].apply(map_to_coarse)

# -----------------------------
# 3. Select 1–3 Samples for Reference and Choose 1 Example for 1-Shot
# -----------------------------
sampled_examples = test_df.sample(3, random_state=42).to_dict(orient="records")
one_shot_example = sampled_examples[0]  # use the first as the 1-shot example

print("📌 Selected Example for 1-Shot Prompting:\n")
print(f"Q: {one_shot_example['question']}\nA: {one_shot_example['coarse_label']}\n")

# -----------------------------
# 4. Build Definition-Enhanced 1-Shot Prompt
# -----------------------------
def build_1shot_prompt(question, example):
    return (
        "You are an expert travel question classifier.\n"
        "Classify each question into one of these categories:\n"
        "TRAVEL: transportation, routes, travel planning, tours.\n"
        "STAY: hotels, accommodation, lodging.\n"
        "EXPERIENCE: activities, food, entertainment, local culture.\n"
        "INFORMATION: weather, safety, or general travel guidance.\n\n"
        f"Example:\nQ: {example['question']}\nA: {example['coarse_label']}\n\n"
        f"Now classify the following question:\nQ: {question}\nA:"
    )

# -----------------------------
# 5. Batched Inference for Speed
# -----------------------------
def run_1shot_inference(df, example, batch_size=12):
    predictions = []
    for i in tqdm(range(0, len(df), batch_size), desc="Generating 1-Shot Predictions"):
        batch_questions = df["question"].iloc[i:i+batch_size].tolist()
        prompts = [build_1shot_prompt(q, example) for q in batch_questions]

        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=15,
                temperature=0.3,
                top_p=0.9,
                do_sample=True,
                repetition_penalty=1.0
            )

        decoded = [tokenizer.decode(o, skip_special_tokens=True).upper() for o in outputs]

        for r in decoded:
            match = re.search(r"\b(TRAVEL|STAY|EXPERIENCE|INFORMATION)\b", r)
            label = match.group(1) if match else "INFORMATION"
            predictions.append(label)

        del inputs, outputs
        torch.cuda.empty_cache()
        gc.collect()

    return predictions

# -----------------------------
# 6. Run 1-Shot Prompting
# -----------------------------
test_df["predicted_label_1shot"] = run_1shot_inference(test_df, one_shot_example)

# -----------------------------
# 7. Evaluate Performance
# -----------------------------
accuracy = accuracy_score(test_df["coarse_label"], test_df["predicted_label_1shot"])
print(f"\n✅ 1-Shot Accuracy on Test Dataset: {round(accuracy, 2)}")

print("\nClassification Report:\n")
print(classification_report(
    test_df["coarse_label"], test_df["predicted_label_1shot"], zero_division=0
))

print("\nSample Predictions:\n")
print(test_df.sample(10)[["question", "coarse_label", "predicted_label_1shot"]])


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

📌 Selected Example for 1-Shot Prompting:

Q: Does Uber run in Chester?
A: TRAVEL



Generating 1-Shot Predictions: 100%|██████████| 59/59 [01:13<00:00,  1.25s/it]


✅ 1-Shot Accuracy on Test Dataset: 0.67

Classification Report:

              precision    recall  f1-score   support

  EXPERIENCE       0.00      0.00      0.00       103
 INFORMATION       0.00      0.00      0.00        13
        STAY       0.00      0.00      0.00       112
      TRAVEL       0.67      1.00      0.81       472

    accuracy                           0.67       700
   macro avg       0.17      0.25      0.20       700
weighted avg       0.45      0.67      0.54       700


Sample Predictions:

                                              question coarse_label  \
522  What are the art galleries where you can buy a...       TRAVEL   
189  Is dune dining included in the full board pack...         STAY   
99            What is the nearest airport to Yosemite?       TRAVEL   
131  What are the best places to have vegan food in...   EXPERIENCE   
387   What are the good areas for snorkeling in Egypt?       TRAVEL   
645  What are the Child friendly restaurants in Dub

In [ ]:
# ============================================================
#  3-Shot Prompt on Test Dataset
#
# ============================================================

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from tqdm import tqdm

# ------------------------------------------------------------
# 1. Model setup
# ------------------------------------------------------------
model_name = "unsloth/zephyr-sft"

tokenizer = AutoTokenizer.from_pretrained(model_name)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="cuda"
)
model.eval()

# ------------------------------------------------------------
# 2. Example test dataset
# ------------------------------------------------------------
test_data = {
    "question": [
        "What are the best places to eat sushi in Tokyo?",
        "Is it safe for solo female travellers to go to Morocco?",
        "How can I find cheap hotels in Paris?",
        "Where can I buy local SIM cards in Bali?",
        "What festivals are happening in Spain this summer?",
        "What’s the best transport from airport to city centre in Rome?"
    ],
    "coarse_label": [
        "EXPERIENCE",
        "INFORMATION",
        "STAY",
        "INFORMATION",
        "EXPERIENCE",
        "TRAVEL"
    ]
}
test_df = pd.DataFrame(test_data)

# ------------------------------------------------------------
# 3. Few-shot exemplars + category definitions
# ------------------------------------------------------------
category_definitions = """
TRAVEL: Questions about transportation, routes, tickets, or tours.
STAY: Questions about hotels, accommodation, or check-in details.
EXPERIENCE: Questions about food, culture, or activities and attractions.
INFORMATION: Questions about safety, weather, visa, or general information.
"""

few_shot_examples = [
    ("What is the best way to travel from Bangkok to Chiang Mai?", "TRAVEL"),
    ("Can you recommend good family hotels in Sydney?", "STAY"),
    ("What are popular street foods in Seoul?", "EXPERIENCE")
]

# ------------------------------------------------------------
# 4. Prompt builder function
# ------------------------------------------------------------
def build_3shot_prompt(question: str) -> str:
    examples = "\n".join([f"Q: {q}\nA: {a}" for q, a in few_shot_examples])
    prompt = (
        f"You are a professional travel question classifier.\n"
        f"Each question belongs to one of four categories:\n{category_definitions}\n\n"
        f"Here are 3 examples:\n{examples}\n\n"
        f"Now, classify this question strictly into one of these categories:\n"
        f"Q: {question}\nA:"
    )
    return prompt

# ------------------------------------------------------------
# 5. Batched inference for GPU speed optimization
# ------------------------------------------------------------
batch_size = 2
predictions = []

for i in tqdm(range(0, len(test_df), batch_size), desc="Generating predictions"):
    batch_questions = test_df["question"][i:i + batch_size].tolist()
    prompts = [build_3shot_prompt(q) for q in batch_questions]

    # Tokenization with truncation and padding
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    # Efficient generation
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            temperature=0.3,
            top_p=0.9,
            do_sample=True,  # deterministic output for consistent classification
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode and extract prediction
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch_preds = [text.split("A:")[-1].strip().split()[0].upper() for text in decoded]
    predictions.extend(batch_preds)

# ------------------------------------------------------------
# 6. Evaluate accuracy
# ------------------------------------------------------------
test_df["predicted_label"] = predictions
test_df["correct"] = test_df["predicted_label"] == test_df["coarse_label"]

accuracy = test_df["correct"].mean()

print("\n=== 3-Shot Definition-Enhanced Results ===")
print(test_df[["question", "coarse_label", "predicted_label"]])
print(f"\n✅ Accuracy: {accuracy:.2f}")

# ------------------------------------------------------------
# 7. Optional: Cleanup GPU cache
# ------------------------------------------------------------
torch.cuda.empty_cache()


`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

Generating predictions: 100%|██████████| 3/3 [00:03<00:00,  1.03s/it]


=== 3-Shot Definition-Enhanced Results ===
                                            question coarse_label  \
0    What are the best places to eat sushi in Tokyo?   EXPERIENCE   
1  Is it safe for solo female travellers to go to...  INFORMATION   
2              How can I find cheap hotels in Paris?         STAY   
3           Where can I buy local SIM cards in Bali?  INFORMATION   
4  What festivals are happening in Spain this sum...   EXPERIENCE   
5  What’s the best transport from airport to city...       TRAVEL   

  predicted_label  
0      EXPERIENCE  
1     INFORMATION  
2            STAY  
3     INFORMATION  
4      EXPERIENCE  
5          TRAVEL  

✅ Accuracy: 1.00


In [ ]:
# ============================================================
# Supervised Fine-Tuning
#
# ============================================================


# -----------------------------
# 1️⃣ Convert pandas DataFrames to HuggingFace Datasets
# -----------------------------
train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)
test_dataset  = Dataset.from_pandas(test_df)

dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

# -----------------------------
# 2️⃣ Clean dataset and map string labels to integers
# -----------------------------
all_labels = sorted(dataset["train"].unique("label"))
label2id = {label: i for i, label in enumerate(all_labels)}
id2label = {i: label for label, i in label2id.items()}

def clean_dataset(dataset):
    def preprocess_safe(examples):
        questions = [str(q[0]) if isinstance(q, list) else str(q) for q in examples["question"]]
        labels = []
        for l in examples["label"]:
            if isinstance(l, list):
                l = l[0]
            if isinstance(l, str):
                l = label2id[l.strip()]
            labels.append(int(l))
        return {"input_text": questions, "labels": labels}
    for split in ["train", "validation", "test"]:
        dataset[split] = dataset[split].map(preprocess_safe, batched=True)
    return dataset

dataset = clean_dataset(dataset)

# -----------------------------
# 3️⃣ Tokenization
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
max_length = 128

def tokenize_batch(examples):
    return tokenizer(
        examples["input_text"],
        truncation=True,
        padding="max_length",
        max_length=max_length
    )

for split in ["train", "validation", "test"]:
    remove_cols = ["input_text", "question", "label"]
    dataset[split] = dataset[split].map(
        tokenize_batch,
        batched=True,
        remove_columns=[c for c in remove_cols if c in dataset[split].column_names]
    )

# -----------------------------
# 4️⃣ Metrics
# -----------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# -----------------------------
# 5️⃣ Subset for Hyperparameter Search
# -----------------------------
search_train_dataset = dataset["train"].shuffle(seed=42).select(range(min(500, len(dataset["train"]))))
search_val_dataset   = dataset["validation"].shuffle(seed=42).select(range(min(100, len(dataset["validation"]))))

# -----------------------------
# 6️⃣ Hyperparameter Grid (small for speed)
# -----------------------------
lora_r_values = [4, 8, 16]
use_4bit_values = [True, False]
bf16_values = [torch.cuda.is_bf16_supported(), False]

best_f1 = -1
best_config = None
best_model = None

# -----------------------------
# 7️⃣ Grid Search Loop
# -----------------------------
for lora_r, use_4bit, bf16 in itertools.product(lora_r_values, use_4bit_values, bf16_values):
    print(f"\n🔹 Training with Lora_r={lora_r}, 4bit={use_4bit}, bf16={bf16}")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16 if bf16 else torch.float32,
    ) if use_4bit else None

    model = AutoModelForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=len(label2id),
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16 if bf16 else torch.float32,
        device_map="auto"
    )

    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=16,
        target_modules=["query", "value"],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.SEQ_CLS
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    training_args = TrainingArguments(
        output_dir=f"./temp_lora_model_r{lora_r}_4bit{use_4bit}_bf16{bf16}",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        learning_rate=2e-4,
        num_train_epochs=1,
        fp16=False,
        bf16=bf16,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        logging_steps=10,
        report_to="none",
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=search_train_dataset,
        eval_dataset=search_val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
    )

    # Train on small subset
    trainer.train()

    # Evaluate
    metrics = trainer.evaluate()
    print(f"Validation F1: {metrics['eval_f1']:.4f}")

    if metrics['eval_f1'] > best_f1:
        best_f1 = metrics['eval_f1']
        best_config = {"lora_r": lora_r, "use_4bit": use_4bit, "bf16": bf16}
        best_model = model

# -----------------------------
# 8️⃣ Fine-tune best model on full dataset
# -----------------------------
print("\n🎯 Best configuration from subset search:", best_config)

training_args = TrainingArguments(
    output_dir="./final_lora_model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-4,
    num_train_epochs=3,  # full training
    fp16=False,
    bf16=best_config["bf16"],
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
)

trainer = Trainer(
    model=best_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

# -----------------------------
# 9️⃣ Evaluate on Test Set
# -----------------------------
test_metrics = trainer.evaluate(dataset["test"])
print("📊 Test Metrics:", test_metrics)

# -----------------------------
# 🔟 Save Best Model
# -----------------------------
trainer.save_model("./best_travel_lora_model")


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.828600,1.800313,0.240000,0.092903


Validation F1: 0.0929

🎯 Best configuration from subset search: {'lora_r': 8, 'use_4bit': True, 'bf16': True}


/tmp/ipython-input-3104871625.py:187: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.925200,0.795207,0.736667,0.721518
2,0.436200,0.561258,0.830000,0.824116
3,0.550500,0.517276,0.850000,0.846810


📊 Test Metrics: {'eval_loss': 0.5415764451026917, 'eval_accuracy': 0.8385714285714285, 'eval_f1': 0.8310675912378211, 'eval_runtime': 3.6994, 'eval_samples_per_second': 189.221, 'eval_steps_per_second': 23.788, 'epoch': 3.0}
